In [51]:
import rdflib
from rdflib import Namespace , Literal , URIRef
from rdflib.namespace import RDF , RDFS
import SPARQLWrapper
from SPARQLWrapper import SPARQLWrapper, JSON
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

# get the endpoint API
#wikidata_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
artchives_endpoint = "http://artchives.fondazionezeri.unibo.it/sparql"

# bind the uncommon namespaces
wd = Namespace("http://www.wikidata.org/entity/") # remember that a prefix matches a URI until the last slash (or hashtag #)
wdt = Namespace("http://www.wikidata.org/prop/direct/")
art = Namespace("https://w3id.org/artchives/")

# create an empty Graph
g = rdflib.ConjunctiveGraph()

# parse a local RDF file by specifying the format
result = g.parse("artchives.nq", format='nquads')


query_result = g.query(
    """PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    SELECT (SAMPLE(?coll_label) AS ?coll) (SAMPLE(?label) AS ?period_label) ?earliest ?latest
    WHERE {?coll <https://w3id.org/artchives/hasSubjectPeriod> ?period ; rdfs:label ?coll_label .
    ?period rdfs:label ?label .
    OPTIONAL {?coll wdt:P1319 ?earliest}
    OPTIONAL {?coll wdt:P1326 ?latest}
    }
    GROUP BY ?period ?coll ?earliest ?latest
    ORDER BY ?period_label""")



In [60]:
# prepare the csv file
import csv
import pandas as pd

with open('periods_dates.csv', mode='w') as my_file:
    my_writer = csv.writer(my_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
    # write the column names
    my_writer.writerow(['collection', 'period_label', 'earliest', 'latest' ])
    
    # access the rows of the query results
    for coll, label, earliest, latest in query_result:
        # write in the csv
        my_writer.writerow([coll.strip(), label.strip(), earliest.strip(), latest.strip()])

In [61]:
# parse the csv into a dataframe
data = pd.read_csv("periods_dates.csv")
# print the first 59 rows
data.head(60)

,collection,period_label,earliest,latest
0,Fototeca Julian Kliemann,17th century,1985-01-01,2015-01-01
1,Fototeca Zeri,17th century,1947-01-01,1998-01-01
2,"Nachlass Fritz Heinemann, Notizen zur venezian...",17th century,1925-01-01,1970-01-01
3,Archivio Luisa Vertova,17th century,1960-01-01,2018-01-01
4,Luigi Salerno research papers,17th century,1948-01-01,1996-01-01
5,Fototeca Stefano Tumidei,17th century,1984-01-01,2007-01-01
6,Julius S. Held papers,17th century,1918-01-01,1999-01-01
7,Nachlass Wolfgang Lotz,17th century,1936-01-01,1981-01-01
8,"Nachlass Fritz Heinemann, Notizen zur venezian...",mannerism,1925-01-01,1970-01-01
9,Ellis K. Waterhouse notebooks and research files,mannerism,1801-01-01,1987-01-01


In [59]:
#trovare un modo per trovare i due anni, uno earliest e l'altro latest. FATTO BOOM! CE L'ABBIAMO FATTA IL CIELO è AZZURROOO SOPRAAAAAA BERLINO